In [ ]:
"""
__authors__ = "Chenyang Li"
__email__   = ["bjyork0822@gmail.com"]

__date__      = "2020-12-07"
"""

# Forte Tutorial: Running MCSCF in Psi4 and Forte

---

In this tutorial we are going to explore how to perform a MCSCF computation using Psi4 and Forte.

## Tip1: Read the Manual!

Psi4 MCSCF/DETCI: all keywords are listed [here](http://www.psicode.org/psi4manual/master/autodir_options_c/module__detci.html#mcscf).

Forte manual (in forte folder):
  1. `cd docs/sphinx`
  2. `make html`
  3. `cd _build/html` and `open index.html`

## 1. Psi4 MCSCF on Ground-State N2 Cation Using Full-Valence Active Space

### Step 1: Find Active Space

The first question we are facing is how to find the active space.
OK, we know the MO diagrams of N2+ and the full valence contains $2\sigma_g, 2\sigma_u, 3\sigma_g, 1\pi_u, 1\pi_g$, and $3\sigma_u$ orbitals.
However, these symbols are in $D_{\infty h}$ and we are computing the molecule in $D_{2h}$.
Of course, we can use our knowledge of group theory and figure out the corresponding irreps.


Here, I will take a shortcut and put my all trust on Psi4 by running a simple SCF using the minimal basis set (STO-3G).
The logic here is that STO-3G only contains 1s, 2s, and 2p functions of N atom.

In [ ]:
import psi4
import forte
import numpy as np

For convenience, we define a function to perform jobs in Psi4.

In [ ]:
psi4.core.set_output_file('single_point_psi4.out', False)  # pipe output to the file

def run_psi4_energy(job, mol, options, ref_wfn=None, forte_options=None):
    """
    Run a Psi4 calculation.
    :param job: a string to be passed to psi4.energy()
    :param mol: a Psi4 Molecule object
    :param options: a Python dictionary to be passed to psi4.set_options()
    :param ref_wfn: a Psi4 Wavefunction object as reference
    :param forte_options: a Python dictionary stores Forte options
    :return: the energy and the Wavefunction
    """
    psi4.core.set_active_molecule(mol)

    psi4.set_options(options)
    if forte_options is not None:
        psi4.set_module_options('forte', forte_options)
    
    if ref_wfn is None:
        Epsi4, wfn = psi4.energy(job, return_wfn=True)
        # molecule can be directly passed to energy:
        # Epsi4, wfn = psi4.energy(job, molecule=mol, return_wfn=True)
    else:
        Epsi4, wfn = psi4.energy(job, ref_wfn=ref_wfn, return_wfn=True)
    
    return Epsi4, wfn

We then create the moleucle and run a minimal basis calculation.

In [ ]:
def geom_N2p(bond):
    geom = f"""1 2
            N
            N 1 {bond}
            """
    return geom

mol_N2p = psi4.geometry(geom_N2p(1.0))

options = {'basis': 'sto-3g', 'reference': 'rohf', 'freeze_core': 'true'}

Escf, wfn = run_psi4_energy('scf', mol_N2p, options)

We then grab the number of MOs per irrep and the number of frozen-core orbitals per irrep.
In this case, the "frozen-core" will be the `RESTRICTED_DOCC` and the remaining will be the `ACTIVE` for the CASSCF.

In [ ]:
nirrep = wfn.nirrep()

occ_map = {
    "FROZEN_DOCC": wfn.frzcpi(),
    "NMOPI": wfn.nmopi(),
    "DOCCPI": wfn.doccpi(),
    "SOCCPI": wfn.soccpi()
}    

print("ROHF with Minimal Basis Set")
print(f"Number of irreps: {nirrep}")
for k, v in occ_map.items():
    print(f"{k:12}: {v.to_tuple()}")


# compute active space
occ_map['ACTIVE'] = occ_map['NMOPI'] - occ_map['FROZEN_DOCC']
occ_map['RESTRICTED_DOCC'] = occ_map['FROZEN_DOCC']

print("\nSuggested Active Space")
print(f"RESTRICTED_DOCC: {occ_map['RESTRICTED_DOCC'].to_tuple()}")
print(f"ACTIVE         : {occ_map['ACTIVE'].to_tuple()}")

### Step 2: Run Psi4 MCSCF

This step should be fairly straightforward and there are many examples in Psi4 and Forte.
In particular, check the `dsrg-mrpt2-6` test case in Forte.

In [ ]:
options = {
    'basis': 'cc-pvdz',
    'reference': 'rohf',
    'scf_type': 'df',
    'maxiter': 100,
    'e_convergence': 8,
    'd_convergence': 6,
    'restricted_docc': [1, 0, 0, 0, 0, 1, 0, 0],
    'active': [2, 0, 1, 1, 0, 2, 1, 1],
    'mcscf_maxiter': 100,
    'mcscf_type': 'df',
    'mcscf_e_convergence': 8,
    'mcscf_r_convergence': 7,
    'mcscf_diis_start': 6
}

In [ ]:
%%time

Ecas, wfn = run_psi4_energy('casscf', mol_N2p, options)
print(f"Psi4 CASSCF(9e,8o) Energy: {Ecas:.15f}")

assert abs(Ecas - -108.523074985101772) < 1.0e-6

## 2. Forte MCSCF on Ground-State N2 Cation Using Full-Valence Active Space

We then carry out the same computation using Forte. For examples, please check `df-casscf-1` and other `casscf-x` test cases in Forte.

There are two codes in Forte that can achieve MCSCF: `CASSCF` and `MCSCF_TWO_STEP`.
The latter code should have the same convergence behavior when setting `CASSCF_MICRO_MAXITER 1`.

Here, we shall exclusively use the `MCSCF_TWO_STEP` code.

In [ ]:
%%time

psi4.core.set_output_file('single_point_forte.out', False)  # to a new file

foptions = {
    'job_type': 'mcscf_two_step',
    'int_type': 'df',
    'restricted_docc': [1,0,0,0,0,1,0,0],
    'active': [2,0,1,1,0,2,1,1],
    'casscf_ci_solver': 'fci',
    'casscf_maxiter': 100,
    'casscf_e_convergence': 1.0e-8,
    'casscf_g_convergence': 1.0e-8
}

Ecas, wfn = run_psi4_energy('forte', mol_N2p, options, forte_options=foptions)
print(f"Forte CASSCF(9e,8o) Energy: {Ecas:.15f}")

assert abs(Ecas - -108.523074985101772) < 1.0e-6

Hopefully the same energy is obtained and no assertion error is generated.

Let's check the output `single_point.out` for a moment.

We see that in between the solving the CI problem, the orbitals are optimized using L-BFGS (micro iterations).
Each micro iteration contains a JK build.
Therefore, the code assumes building JK is cheap and CI is rather expensive.
This is certainly not always the case.
For very large diradical systems, it might be advantageous to lower the max number of micro iterations by specifying keyword `CASSCF_MICRO_MAXITER`.

## 3. Potential Energy Curve

So far so good? Let us run a potential energy curve and plot it using Psi4.

In [ ]:
bonds = [1.0 + 0.1 * i for i in range(10)] \
        + [2.0 + 0.2 * i for i in range(10)] \
        + [4.0 + 0.5 * i for i in range(5)]

In [ ]:
%%time

psi4.core.set_output_file('curve_psi4_try1.out', False)  # to a new file

options = {
    'basis': 'cc-pvdz',
    'reference': 'rohf',
    'scf_type': 'df',
    'maxiter': 100,
    'e_convergence': 3,  # not good practice, but cooked up for this tutorial
    'd_convergence': 1,  # not good practice, but cooked up for this tutorial
    'restricted_docc': [1, 0, 0, 0, 0, 1, 0, 0],
    'active': [2, 0, 1, 1, 0, 2, 1, 1],
    'mcscf_maxiter': 100,
    'mcscf_type': 'df',
    'mcscf_e_convergence': 8,
    'mcscf_r_convergence': 7,
    'mcscf_diis_start': 6
}

psi4cas_results = []

for r in bonds:
    psi4.core.print_out(f"\n  Setting bond length to {r}")
    mol_N2p = psi4.geometry(geom_N2p(r))
    Ecas, wfn = run_psi4_energy('casscf', mol_N2p, options)
    psi4cas_results.append(Ecas)

for r, e in zip(bonds, psi4cas_results):
    print(f"{r:.2f} {e:18.12f}")

In [ ]:
%matplotlib notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ft2font

mpl.rcParams['pdf.fonttype'] = 42
mpl.rc('font', family='sans-serif') 
mpl.rc('font', serif='Helvetica')

In [ ]:
fig = plt.figure(figsize=(8,5), dpi=100)
ax = plt.subplot(111)

ax.plot(bonds, psi4cas_results)

ax.set_title("Psi4 DF-CASSCF(9e,8o)/cc-pVDZ on N2 Cation")
ax.set_xlim(0.9, 6.1)
ax.set_xlabel('$r$ / $\AA$')
ax.set_ylabel('E / a.u.')

Do you see the kinks around 1.5 A (not so obvious) and 3.6 A (very obvious)? What is going on here?

The clue lies in the output!

## Tip 2: Check the Output File Carefully!

Here is what I find in the output (See the hidden text).

<span style='color:white;background:white'>
Check the <code style='color:white;background:white'>DOCC</code> and <code style='color:white;background:white'>SOCC</code> in ROHF.
    
We know the ground state is doublet Ag ($\Sigma_g^-$).
And the <code style='color:white;background:white'>SOCC</code> changes multiple times along the curve.

Why this matters? At the end, we really cares about CASSCF and ROHF does not matter.

The reason is that the Psi4 MCSCF assumes the same symmetry as the SCF one.

Therefore, we are not actually following the doublet Ag state along the curve.

**A good practice to compute a curve is always specify the <code style='color:white;background:white'>DOCC</code> and <code style='color:white;background:white'>SOCC</code> of SCF and the <code style='color:white;background:white'>REFERENCE_SYM</code> option for MCSCF.**
</span>


We now change the options and recompute the curve.

In [ ]:
psi4.core.set_output_file('curve_psi4_try2.out', False)  # to a new file

options = {
    'basis': 'cc-pvdz',
    'reference': 'rohf',
    'scf_type': 'df',
    'maxiter': 100,
    'e_convergence': 8,
    'd_convergence': 6,
    'docc': [2, 0, 0, 0, 0, 2, 1, 1],  # good practice
    'socc': [1, 0, 0, 0, 0, 0, 0, 0],  # good practice
    'restricted_docc': [1, 0, 0, 0, 0, 1, 0, 0],
    'active': [2, 0, 1, 1, 0, 2, 1, 1],
    'mcscf_maxiter': 100,
    'mcscf_type': 'df',
    'mcscf_e_convergence': 8,
    'mcscf_r_convergence': 7,
    'mcscf_diis_start': 6,
    'reference_sym': 0  # good practice
}

# compute energies
psi4cas_results_socc = []

for r in bonds:
    psi4.core.print_out(f"\n  Setting bond length to {r}")
    mol_N2p = psi4.geometry(geom_N2p(r))
    Ecas, wfn = run_psi4_energy('casscf', mol_N2p, options)
    psi4cas_results_socc.append(Ecas)

In [ ]:
# create a new figure but keep the old data
fig = plt.figure(figsize=(8,5), dpi=100)
ax = plt.subplot(111)
ax.plot(bonds, psi4cas_results, label='first try')

# plot
ax.plot(bonds, psi4cas_results_socc, label='socc specified')

ax.set_title("Psi4 DF-CASSCF(9e,8o)/cc-pVDZ on N2 Cation")
ax.set_xlim(0.9, 6.1)
ax.set_xlabel('$r$ / $\AA$')
ax.set_ylabel('E / a.u.')

ax.legend(loc='lower right')

Now it looks better. But I am still not confortable with the kink at 1.9 A.

What should we do? Perhaps it will be smoothened using finer grid. But if you trust me, I have run 400 points on a 0.01 A grid and the kink is still there.

Check the output, we see the configuration at 2.0 A is drastically different than that at 1.9 A.
Usually, this suggest we are landing on a different state: There might be a strongly interacting excited state there!
To do that, we need to do state-averaged CASSCF or dynamically weighted CASSCF (left as an exercise).

Oh, but wait. I have not heard any avoided crossing for N2 cation.
Could this be realted to [this post](http://forum.psicode.org/t/detci-finds-an-extra-excited-state/1729)?

## Tip 3: Do Literature Search and ALWAYS Question the Results

<span style='color:white;background:white'>
Indeed, this is a Psi4 problem. The state we have after 2.0 A is actually a sextet state. Try that!

So after all, should we trust Psi4 CASSCF?
I guess it depends.
If anything suspicious appears, we should ALWAYS compare with other codes (MOLPRO, MOLCAS, ORCA, GAMESS, etc.).
Thankfully, MCSCF is also available in Forte and there is minimal effort to change the Psi4 MCSCF input to Forte's.
</span>

## 3. Try Better CASSCF Starting Orbitals

The last thing I would like to show using Psi4's MCSCF is to pass different orbitals.

There are two situations that we are intersted in.

- 1. Different orbitals but **same** geometry

    This is the common case where the SCF orbitals may not be ideal.
    For example, suppose we want to compute the singlet state of a diradical.
    The CASSCF(2,2) orbitals should look like the triplet ROHF orbitals more than the singlet RHF orbitals.
    As such, we might just pass the triplet ROHF orbitals as starting guess for singlet CASSCF(2,2).
    
    For other cases, we might start from MP2 natural orbitals, CCSD natural orbitals,
    or any other type of orbitals preferred.
    
    It is also very easy to do achieve this by simply passing the previous Psi4 Wavefunction as reference wave function for the current MCSCF.
    ```python
    Escf, wfn = energy('rohf', return_wfn=True)  # triplet ROHF
    energy('casscf', ref_wfn=wfn)                # singlet CASSCF(2,2)
    ```

- 2. Different orbitals and **different** geometries

    This is encountered for a scan of the potential energy surface (like we did for the N2 cation).

    However, we cannot directly use the orbitals from a previous geometry because the AO integrals are different (between previous and current geometries).
    Psi4 is not smart enough to orthogonalize the orbitals for us.
    You should try to pass the Wavefunction of a previous geometry to the current geometry and check the energy.
    (This is allowed but the energy is WRONG!)
    
    I have a script to achieve this orthogonalization and it is available in Forte (`proc/orthogonalize_orbitals`).
    Here is the doctring of the `ortho_orbs_psi4` function that we will use.

```python
def ortho_orbs_psi4(wfn1, wfn2, semi=True):
    """
    Make orbitals of geometry 1 (old) orthonormal with the basis
    from geometry 2 (current):
    (C1)^T S2 C1 = 1, where C1 is the CASSCF orbitals at geometry 1
    and S2 is the SO overlap matrix at geometry 2.

    :param wfn1: Psi4 Wavefunction from geometry 1
    :param wfn2: Psi4 Wavefunction from geometry 2
    :param semi: Semicanonicalize resulting orbitals
    :return: orthogonal orbital coefficients

    Example:
        molecule HF {
        F
        H 1 R
        }

        set {
          basis cc-pvdz
          reference rhf
          restricted_docc [2,0,1,1]
          active [2,0,0,0]
        }

        HF.R = 1.0
        Ecas, wfn = energy('casscf', return_wfn=True)

        HF.R = 1.1
        Escf, wfnSCF = energy('scf', return_wfn=True)
        wfnSCF.Ca().copy(ortho_orbs_psi4(wfn, wfnSCF))
        Ecas = energy('casscf', ref_wfn=wfnSCF)
    """
```

Now we go back to the potential energy curve of N2 cation and try to feed the orbitals from the previous geometry.

In [ ]:
from forte.proc.orthogonalize_orbitals import ortho_orbs_psi4

psi4.core.set_output_file('curve_psi4_try3.out', False)  # to a new file

psi4cas_results_read = []

wfn = None

for r in bonds[5:-5]:
    psi4.core.print_out(f"\n  Setting bond length to {r}")
    mol_N2p = psi4.geometry(geom_N2p(r))

    if wfn is None:
        Ecas, wfn = run_psi4_energy('casscf', mol_N2p, options)
    else:
        Escf, wfnSCF = run_psi4_energy('scf', mol_N2p, options)
        wfnSCF.Ca().copy(ortho_orbs_psi4(wfn, wfnSCF))
        Ecas, wfn = run_psi4_energy('casscf', mol_N2p, options, wfnSCF)
    
    psi4cas_results_read.append(Ecas)

In [ ]:
# create a new figure but keep the old data
fig = plt.figure(figsize=(8,5), dpi=100)
ax = plt.subplot(111)
ax.plot(bonds, psi4cas_results, label='first try')
ax.plot(bonds, psi4cas_results_socc, label='socc specified')

# plot
ax.plot(bonds[5:-5], psi4cas_results_read,
        color='red', label='read')

ax.set_title("Psi4 DF-CASSCF(9e,8o)/cc-pVDZ on N2 Cation")
ax.set_xlim(0.9, 6.1)
ax.set_xlabel('$r$ / $\AA$')
ax.set_ylabel('E / a.u.')

ax.legend(loc='lower right')

Nothing changes. This is what we expect if you find the previous hidden text!

Now, let's try with Forte.

When a Psi4 Wavefunction is passed to Forte, it will check if the MO overlap using the orbital coefficients from the Wavefunction and the AO overlap integrals from the current active the molecule.
In the MO overlap is not identity, it will run an SCF calculation and perform a similar orthogonalization procedure as we have seen previously.

In [ ]:
psi4.core.set_output_file('curve_forte.out', False)  # to a new file

options = {
    'basis': 'cc-pvdz',
    'reference': 'rohf',
    'scf_type': 'df',
    'maxiter': 100,
    'e_convergence': 8,
    'd_convergence': 6,
    'docc': [2, 0, 0, 0, 0, 2, 1, 1],
    'socc': [1, 0, 0, 0, 0, 0, 0, 0]
}

foptions = {
    'job_type': 'mcscf_two_step',
    'int_type': 'df',
    'restricted_docc': [1,0,0,0,0,1,0,0],
    'active': [2,0,1,1,0,2,1,1],
    'casscf_ci_solver': 'fci',
    'casscf_maxiter': 100,
    'casscf_e_convergence': 1.0e-8,
    'casscf_g_convergence': 1.0e-8,
    'print': 1,
    'fci_maxiter': 200
}

forte_cas_results = []

wfn = None

for r in bonds:
    psi4.core.print_out(f"\n  Setting bond length to {r}")
    mol_N2p = psi4.geometry(geom_N2p(r))
    mol_N2p.print_out();

    if wfn is None:
        Escf, wfn = run_psi4_energy('scf', mol_N2p, options)
    
    Ecas, wfn = run_psi4_energy('forte', mol_N2p, options,
                                ref_wfn=wfn, forte_options=foptions)
    
    # another way to grab energy, useful for multiple states / roots
    root0 = psi4.variable("ENERGY ROOT 0 2Ag")
    
    forte_cas_results.append(root0)

In [ ]:
# create a new figure
fig = plt.figure(figsize=(8,5), dpi=100)
ax = plt.subplot(111)

ax.plot(bonds, psi4cas_results, label='psi4 first try')
ax.plot(bonds, psi4cas_results_socc, label='psi4 socc specified')

# plot
ax.plot(bonds, forte_cas_results, label='forte')

ax.set_title("DF-CASSCF(9e,8o)/cc-pVDZ on N2 Cation")
ax.set_xlim(0.9, 6.1)
ax.set_xlabel('$r$ / $\AA$')
ax.set_ylabel('E / a.u.')

ax.legend(loc='lower right')

**Hooray! No kint at 1.9 A!**

**Exercise**

Compute the potential energy curves for the lowest two $^1 \Sigma^+$ state of LiF using equal-weighted state-averaged CASSCF with a full-valence active space and the cc-pVDZ basis set.
Try both Psi4 and Forte!

## 4. Frozen-Core Approximation in MCSCF

We do not usually freeze any orbitals in MCSCF computations.
By freezing orbitals in MCSCF, we mean we want to exclude some orbitals from the MCSCF orbital optimization.
If the frozen orbitals come from Hartree-Fock, they will remain the Hartree-Fock orbitals;
if they are MP2 natural orbitals, they will remain the MP2 NOs.
As we see here, we need to specifically mention what **type** of orbitals are frozen, as well as what are they (i.e., 1s-like orbitals, etc.).

With that being said, here is how we can do that in Forte.

Going back to our N2 cation example, we now want to compute N2 cation at 1.0 A with frozen-core approximation.
The core orbitals are the MOs mainly built from 1s of N atom.

- For full-valence active space CAS(9e,8o), this means we no longer have `restricted_docc`.
- Optionally, we could use CAS(6,6) and keep the $2\sigma_g$ and $2\sigma_u$ as `restricted_docc` (left as exercise).

In [ ]:
psi4.core.set_output_file('single_point_forte_fc.out', False)

mol_N2p = psi4.geometry(geom_N2p(1.0))

foptions = {
    'job_type': 'mcscf_two_step',
    'int_type': 'df',
    'frozen_docc': [1,0,0,0,0,1,0,0],
    'restricted_docc': [0,0,0,0,0,0,0,0],
    'active': [2,0,1,1,0,2,1,1],
    'casscf_ci_solver': 'fci',
    'casscf_maxiter': 100,
    'casscf_e_convergence': 1.0e-8,
    'casscf_g_convergence': 1.0e-8
}

Ecas, wfn = run_psi4_energy('forte', mol_N2p, options, forte_options=foptions)
print(f"Forte FC-CASSCF(9e,8o) Energy: {Ecas:.15f}")

assert abs(Ecas - -108.523032682199528) < 1.0e-6

Comparing to the all-electron calculation we did in Section 1, the energy difference is (-108.52303268 - -108.52307498) = 4.230e-5.

This is the typical value (1e-6 to 1e-5) from frozen-core approximation for first-row diatomics.

## 5. MCSCF Analytic Energy Gradient

As a final remark, the MCSCF analytic energy gradient is available in Forte.
We shall compare the optimized structure with the finite-difference result from Psi4.

We optimize the geometry of N2 cation at the CASSCF(9e,8o)/cc-pVDZ level of theory.
We do not freeze any orbitals because the I only coded the RHF-based CP-SCF equations for frozen orbitals.

**Warning 1**: There is a small discrepancy between analytic and finite-difference results using density-fitted integrals. I think this is a Psi4 DF derivative integral problem but I am not 100% sure.

**Warning 2**: MCSCF analytic gradient with frozen orbitals are only available for singlet, because I only coded up the RHF-based CP-SCF equations.

**Warning 3**: State-averaged CASSCF analytic gradient is not currently available in Forte.

In [ ]:
%%time

# Optimize using Forte

psi4.core.set_output_file('optimize_forte.out', False)

mol_N2p = psi4.geometry(geom_N2p(1.1))
psi4.core.set_active_molecule(mol_N2p)

options = {
    'basis': 'cc-pvdz',
    'reference': 'rohf',
    'scf_type': 'df',
    'maxiter': 100,
    'e_convergence': 8,
    'd_convergence': 6,
    'docc': [2, 0, 0, 0, 0, 2, 1, 1],
    'socc': [1, 0, 0, 0, 0, 0, 0, 0]
}

foptions = {
    'job_type': 'mcscf_two_step',
    'int_type': 'df',
    'frozen_docc': [0,0,0,0,0,0,0,0],
    'restricted_docc': [1,0,0,0,0,1,0,0],
    'active': [2,0,1,1,0,2,1,1],
    'casscf_ci_solver': 'fci',
    'casscf_maxiter': 100,
    'casscf_e_convergence': 1.0e-8,
    'casscf_g_convergence': 1.0e-8
}

psi4.set_options(options)
psi4.set_module_options('forte', foptions)

Eforte, wfn = psi4.optimize('forte', molecule=mol_N2p, return_wfn=True)

print(wfn.molecule().to_string(dtype='psi4', units='angstrom'))

In [ ]:
%%time

# now finite-difference using Psi4

psi4.core.set_output_file('optimize_psi4.out', False)

mol_N2p = psi4.geometry(geom_N2p(1.1))
psi4.core.set_active_molecule(mol_N2p)

options = {
    'basis': 'cc-pvdz',
    'reference': 'rohf',
    'scf_type': 'df',
    'maxiter': 100,
    'e_convergence': 8,
    'd_convergence': 6,
    'docc': [2, 0, 0, 0, 0, 2, 1, 1],  # good practice
    'socc': [1, 0, 0, 0, 0, 0, 0, 0],  # good practice
    'restricted_docc': [1, 0, 0, 0, 0, 1, 0, 0],
    'active': [2, 0, 1, 1, 0, 2, 1, 1],
    'mcscf_maxiter': 100,
    'mcscf_type': 'df',
    'mcscf_e_convergence': 8,
    'mcscf_r_convergence': 7,
    'mcscf_diis_start': 6,
    'reference_sym': 0,  # good practice
    'findif__points': 5  # another way to specify moldule specific options using double underscore
}

psi4.set_options(options)

Epsi4, wfn = psi4.optimize('casscf', molecule=mol_N2p, return_wfn=True)

print(wfn.molecule().to_string(dtype='psi4', units='angstrom'))

As Warning 1 suggested, we do observe the small discrepancies here.
The difference is 0.00009 A, which might be good enough for publishing.